> # RNN啊
我们要学习的模型是LSTM,是RNN的升华版
这是人工智能的范畴了
为什么我这次notebook要写在kaggle上呢
因为我电脑配置的是pytorch，当我尝试配置tensorflow发现一个问题
各位电脑可能不支持cuda
一个加速显卡优化计算的东西
即使可以
我们要统一tensoflow版本，然后各位电脑配置也很难算的快
所以我们干脆就在kaggle上面跑了，各位也方便
我们搭最简单的模型就够了，很深的网络是为了抽取很抽象的特征，我们的特征其实已经挺抽象了

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_log_error

from datetime import datetime
from datetime import timedelta

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 看一下数据到底是什么样

In [ ]:
#######################读取最基本的训练数据
##############################################为了和之前训练的模型相匹配，我们就根据3.24前的数据训练

In [ ]:
train_df = gpd.read_file("/kaggle/input/covid19-global-forecasting-week-4/train.csv")

In [ ]:
train_df.head()
#####################为什么地区需要有两个数据列，是冗余的部分，可以校正

In [ ]:
train_df = gpd.read_file("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
train_df["ConfirmedCases"] = train_df["ConfirmedCases"].astype("float")
train_df["Fatalities"] = train_df["Fatalities"].astype("float")
train_df["Country_Region"] = [ row.Country_Region.replace("'","").strip(" ") if row.Province_State=="" else str(row.Country_Region+"_"+row.Province_State).replace("'","").strip(" ") for idx,row in train_df.iterrows()]

In [ ]:
train_df[train_df["Country_Region"]=="China"] #################国家和地区在一起了

In [ ]:
#调用上传的丰富数据库，含有其他团队整理的信息
extra_data_df = gpd.read_file("/kaggle/input/covid19-enriched-dataset-week-2/enriched_covid_19_week_2.csv")
extra_data_df["Country_Region"] = [country_name.replace("'","") for country_name in extra_data_df["Country_Region"]]
############################类型转换，所有数据使用前都进行一次数据转换，防止奇奇怪怪的类型错误
extra_data_df["restrictions"] = extra_data_df["restrictions"].astype("int")
extra_data_df["quarantine"] = extra_data_df["quarantine"].astype("int")
extra_data_df["schools"] = extra_data_df["schools"].astype("int")
extra_data_df["total_pop"] = extra_data_df["total_pop"].astype("float")
extra_data_df["density"] = extra_data_df["density"].astype("float")
extra_data_df["hospibed"] = extra_data_df["hospibed"].astype("float")
extra_data_df["lung"] = extra_data_df["lung"].astype("float")
############################将每一个数据列与该数据列的最大值进行一个比值，做一个正则化处理
extra_data_df["total_pop"] = extra_data_df["total_pop"]/max(extra_data_df["total_pop"])
extra_data_df["density"] = extra_data_df["density"]/max(extra_data_df["density"])
extra_data_df["hospibed"] = extra_data_df["hospibed"]/max(extra_data_df["hospibed"])
extra_data_df["lung"] = extra_data_df["lung"]/max(extra_data_df["lung"])
extra_data_df["age_100+"] = extra_data_df["age_100+"].astype("float")
extra_data_df["age_100+"] = extra_data_df["age_100+"]/max(extra_data_df["age_100+"])
###########################################################只提取必要的属性列
extra_data_df = extra_data_df[["Country_Region","Date","restrictions","quarantine","schools","hospibed","lung","total_pop","density","age_100+"]]
extra_data_df.head()
############################################约束措施，隔离措施，收治率，肺病率，总人口，密度，百岁老人比例
#####################之前美国欧洲不都是养老院集体失守

In [ ]:
extra_data_df.head()

In [ ]:

train_df = train_df.merge(extra_data_df, how="left", on=['Country_Region','Date']).drop_duplicates()
train_df.head()
######################合并总的数据

####################新的数据并不完整

In [ ]:
##########进一步处理
######对每一个国家分别做处理
######因为额外数据只提供到3月25号之前，我们需要填补缺失值

In [ ]:
for country_region in train_df.Country_Region.unique():
    query_df = train_df.query("Country_Region=='"+country_region+"' and Date=='2020-03-25'")
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"total_pop"] = query_df.total_pop.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"hospibed"] = query_df.hospibed.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"density"] = query_df.density.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"lung"] = query_df.lung.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"age_100+"] = query_df["age_100+"].values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"restrictions"] = query_df.restrictions.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"quarantine"] = query_df.quarantine.values[0]
    train_df.loc[(train_df["Country_Region"]==country_region) & (train_df["Date"]>"2020-03-25"),"schools"] = query_df.schools.values[0]

In [ ]:
#####增补数据中有很多国家有空缺，针对空缺的国家我们用世界的中位数去替代

In [ ]:
############################这里在计算世界的均值
median_pop = np.median(extra_data_df.total_pop)
median_hospibed = np.median(extra_data_df.hospibed)
median_density = np.median(extra_data_df.density)
median_lung = np.median(extra_data_df.lung)
median_centenarian_pop = np.median(extra_data_df["age_100+"])
#######################################额外的数据库中哪里有缺失值
print("The missing countries/region are:")
#########################################遍布每一个原始数据集自己的数据
#########################################在额外数据中寻找是否存在
for country_region in train_df.Country_Region.unique():
    if extra_data_df.query("Country_Region=='"+country_region+"'").empty:
        print(country_region)
        train_df.loc[train_df["Country_Region"]==country_region,"total_pop"] = median_pop
        train_df.loc[train_df["Country_Region"]==country_region,"hospibed"] = median_hospibed
        train_df.loc[train_df["Country_Region"]==country_region,"density"] = median_density
        train_df.loc[train_df["Country_Region"]==country_region,"lung"] = median_lung
        train_df.loc[train_df["Country_Region"]==country_region,"age_100+"] = median_centenarian_pop
        train_df.loc[train_df["Country_Region"]==country_region,"restrictions"] = 0
        train_df.loc[train_df["Country_Region"]==country_region,"quarantine"] = 0
        train_df.loc[train_df["Country_Region"]==country_region,"schools"] = 0

In [ ]:
#####采取和之前lightGBM一样的方法，构造属性列，趋势数据，还记得吗
#####当时人为间隔了数据列，人为模拟控制了爆发期ar
#####当时如果能理解这个应该很简单

In [ ]:
trend_df = pd.DataFrame(columns={"infection_trend","fatality_trend","quarantine_trend","school_trend","total_population","expected_cases","expected_fatalities"})

In [ ]:
train_df.info

# # # *注意啊，我们调整了训练时间，不仅模型变化了，我们的训练范围也变化了*************

In [ ]:
#丢掉了第一天的数据，因为这样刚好就能是7天的一个周期了
#根据过去三周的趋势以及基础设施条件去预测最后一天的情况
#跟新啊：为了配合第一周的预测
#我们选用的训练数据是1.22到3.25
#我们的周期选用的是7天


model_train_df = train_df.query("Date>'2020-01-22'and Date<'2020-03-24'")
days_in_sequence = 5

trend_list = []

with tqdm(total=len(list(model_train_df.Country_Region.unique()))) as pbar:
    for country in model_train_df.Country_Region.unique():
        for province in model_train_df.query(f"Country_Region=='{country}'").Province_State.unique():
            province_df = model_train_df.query(f"Country_Region=='{country}' and Province_State=='{province}'")
            
            for i in range(0,len(province_df),int(days_in_sequence/3)):
                if i+days_in_sequence<=len(province_df):
                    ###########################这一步很重要
                    ###########################在做什么呢？
                    ###########################在做一个滑动的序列，以天为单位滑动
                    ##########################每次滚动一周
                    infection_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].ConfirmedCases.values]
                    fatality_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].Fatalities.values]
                    restriction_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].restrictions.values]
                    quarantine_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].quarantine.values]
                    school_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].schools.values]
                    
                    
                    #########################下面就不是时间序列了，不需要滚
                    total_population = float(province_df.iloc[i].total_pop)
                    density = float(province_df.iloc[i].density)
                    hospibed = float(province_df.iloc[i].hospibed)
                    lung = float(province_df.iloc[i].lung)
                    centenarian_pop = float(province_df.iloc[i]["age_100+"])

                    
                    #######################滚动每个时间点预计的死亡确诊人数
                    expected_cases = float(province_df.iloc[i+days_in_sequence-1].ConfirmedCases)
                    expected_fatalities = float(province_df.iloc[i+days_in_sequence-1].Fatalities)

                    ##########################
                    trend_list.append({"infection_trend":infection_trend,
                                     "fatality_trend":fatality_trend,
                                     "restriction_trend":restriction_trend,
                                     "quarantine_trend":quarantine_trend,
                                     "school_trend":school_trend,
                                     "demographic_inputs":[total_population,density,hospibed,lung,centenarian_pop],
                                     "expected_cases":expected_cases,
                                     "expected_fatalities":expected_fatalities})
        pbar.update(1)  #########################这条代码就是显示下面的进度
trend_df = pd.DataFrame(trend_list)

In [ ]:
###############################接下来我们要做一组显示效果的测试集数据

In [ ]:
#丢掉了第一天的数据，因为这样刚好就能是7天的一个周期了
#根据过去三周的趋势以及基础设施条件去预测最后一天的情况
#跟新啊：为了配合第一周的预测
#我们选用的训练数据是1.22到3.25
#我们的周期选用的是7天


model_test_df = train_df.query("Date>'2020-03-26'and Date<'2020-04-14' and Country_Region =='Afghanistan'")
days_in_sequence = 5

test_trend_list = []

with tqdm(total=len(list(model_test_df.Country_Region.unique()))) as pbar:
    for country in model_test_df.Country_Region.unique():
        for province in model_test_df.query(f"Country_Region=='{country}'").Province_State.unique():
            province_df = model_test_df.query(f"Country_Region=='{country}' and Province_State=='{province}'")
            
            for i in range(0,len(province_df),int(days_in_sequence/3)):
                if i+days_in_sequence<=len(province_df):
                    ###########################这一步很重要
                    ###########################在做什么呢？
                    ###########################在做一个滑动的序列，以天为单位滑动
                    ##########################每次滚动一周
                    infection_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].ConfirmedCases.values]
                    fatality_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].Fatalities.values]
                    restriction_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].restrictions.values]
                    quarantine_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].quarantine.values]
                    school_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].schools.values]
                    
                    
                    #########################下面就不是时间序列了，不需要滚
                    total_population = float(province_df.iloc[i].total_pop)
                    density = float(province_df.iloc[i].density)
                    hospibed = float(province_df.iloc[i].hospibed)
                    lung = float(province_df.iloc[i].lung)
                    centenarian_pop = float(province_df.iloc[i]["age_100+"])

                    
                    #######################滚动每个时间点预计的死亡确诊人数
                    expected_cases = float(province_df.iloc[i+days_in_sequence-1].ConfirmedCases)
                    expected_fatalities = float(province_df.iloc[i+days_in_sequence-1].Fatalities)

                    ##########################
                    test_trend_list.append({"infection_trend":infection_trend,
                                     "fatality_trend":fatality_trend,
                                     "restriction_trend":restriction_trend,
                                     "quarantine_trend":quarantine_trend,
                                     "school_trend":school_trend,
                                     "demographic_inputs":[total_population,density,hospibed,lung,centenarian_pop],
                                     "expected_cases":expected_cases,
                                     "expected_fatalities":expected_fatalities})
        pbar.update(1)  #########################这条代码就是显示下面的进度
test_trend_df = pd.DataFrame(test_trend_list)

In [ ]:
model_train_df.head()

In [ ]:
model_test_df.head()

In [ ]:
############################每一个trend都是对接下来21天发展的估计，demographic_inputs是我们的其他变量
############################

In [ ]:
len(model_test_df)

In [ ]:
len(test_trend_df)

In [ ]:
        ##################################得到的具体分组数据，有趋势，有硬件条件
                #########################很关键啊，我们这一份的数据每一行都是当日确诊人数以及感染确诊趋势，以及硬性条件趋势
            #########################################前面的趋势，是连续13天的趋势，最后是第十四天的确诊以及致死病例
            #########################################那这就存在一个很好的预测形式

In [ ]:
trend_df["temporal_inputs"] = [np.asarray([trends["infection_trend"],trends["fatality_trend"],trends["restriction_trend"],trends["quarantine_trend"],trends["school_trend"]]) for idx,trends in trend_df.iterrows()]  
#########################################当然要打乱顺序，数据之间不能有联系

In [ ]:
trend_df = shuffle(trend_df)

In [ ]:
test_trend_df["temporal_inputs"] = [np.asarray([trends["infection_trend"],trends["fatality_trend"],trends["restriction_trend"],trends["quarantine_trend"],trends["school_trend"]]) for idx,trends in test_trend_df.iterrows()]

In [ ]:
test_trend_df             ######################这是我们的测试集合

In [ ]:
#######################temporal_inputs是所有输入的集合啊，所有趋势数据的集合

######################################全是0的数据我们就不需要了，趋势是0，我们选择保留5份

# ######################重要，我们要转变数据输入形似

In [ ]:
sequence_length = 4

In [ ]:
training_item_count = int(len(trend_df))
# validation_item_count = len(trend_df)-int(len(trend_df)*training_percentage)
training_df = trend_df

In [ ]:
X_temporal_train = np.asarray(np.transpose(np.reshape(np.asarray([np.asarray(x) for x in training_df["temporal_inputs"].values]),(training_item_count,5,sequence_length)),(0,2,1) )).astype(np.float32)
X_demographic_train = np.asarray([np.asarray(x) for x in training_df["demographic_inputs"]]).astype(np.float32)
Y_cases_train = np.asarray([np.asarray(x) for x in training_df["expected_cases"]]).astype(np.float32)
Y_fatalities_train = np.asarray([np.asarray(x) for x in training_df["expected_fatalities"]]).astype(np.float32)

In [ ]:
########################################比如我们调用

In [ ]:
test_trend_df

In [ ]:
test_item_count = int(len(test_trend_df))
# validation_item_count = len(trend_df)-int(len(trend_df)*training_percentage)
test_df = test_trend_df

In [ ]:
X_temporal_test = np.asarray(np.transpose(np.reshape(np.asarray([np.asarray(x) for x in test_df["temporal_inputs"].values]),(test_item_count,5,sequence_length)),(0,2,1) )).astype(np.float32)
X_demographic_test = np.asarray([np.asarray(x) for x in test_df["demographic_inputs"]]).astype(np.float32)
Y_cases_test = np.asarray([np.asarray(x) for x in test_df["expected_cases"]]).astype(np.float32)
Y_fatalities_test = np.asarray([np.asarray(x) for x in test_df["expected_fatalities"]]).astype(np.float32)

# ###########################下面这几段无视

In [ ]:
# training_item_count = int(len(trend_df)*training_percentage)
# validation_item_count = len(trend_df)-int(len(trend_df)*training_percentage)
# training_df = trend_df[:training_item_count]
# validation_df = trend_df[training_item_count:]

In [ ]:
# X_temporal_train = np.asarray(np.transpose(np.reshape(np.asarray([np.asarray(x) for x in training_df["temporal_inputs"].values]),(training_item_count,5,sequence_length)),(0,2,1) )).astype(np.float32)
# X_demographic_train = np.asarray([np.asarray(x) for x in training_df["demographic_inputs"]]).astype(np.float32)
# Y_cases_train = np.asarray([np.asarray(x) for x in training_df["expected_cases"]]).astype(np.float32)
# # Y_fatalities_train = np.asarray([np.asarray(x) for x in training_df["expected_fatalities"]]).astype(np.float32)

In [ ]:
#############################################################################注意啊注意，我们最终的效果不是源自这个测试集

In [ ]:
# X_temporal_test = np.asarray(np.transpose(np.reshape(np.asarray([np.asarray(x) for x in validation_df["temporal_inputs"]]),(validation_item_count,5,sequence_length)),(0,2,1)) ).astype(np.float32)
# X_demographic_test = np.asarray([np.asarray(x) for x in validation_df["demographic_inputs"]]).astype(np.float32)
# Y_cases_test = np.asarray([np.asarray(x) for x in validation_df["expected_cases"]]).astype(np.float32)
# Y_fatalities_test = np.asarray([np.asarray(x) for x in validation_df["expected_fatalities"]]).astype(np.float32)

# 3. 训练模型

这就是我们的模型


###########################假装这里有张图片

In [ ]:
#temporal input branch
###############时间序列数据输入分支
##################逐个详解

##########################这是我们的输入,看上面我们的输入形似，序列数据的输入形式
 # 这里是作为Sequential模型的第一层所以指定input_shape参数，后面加的层不需要这个
    # 这里的input_shape是两个元素的，第一个代表每个输入的样本序列长度，第二个元素代表
    # 每个序列里面的1个元素具有多少个输入数据。例如，LSTM处理的序列长度为20，每个时间
    # 步即序列的元素是由两个维度组成，那么这个参数设置为(20, 5)

    # 输入是固定格式。无论怎么改网络结构我们都是这样子输入
# temporal_input_layer = Input(shape=(sequence_length,5))
####################################输出的单元数目
####################################return_sequences=True返回LSTM中间状态

####################################这和pytorch中的dropout应该一样，都是短接网络，防止过拟合

temporal_input_layer=Input(shape=(sequence_length,5))
temporal_dense=layers.Dense(16)(temporal_input_layer)
main_rnn_layer = layers.LSTM(64, return_sequences=True, recurrent_dropout=0.2)(temporal_dense)



# main_rnn_layer = layers.LSTM(64, return_sequences=True, recurrent_dropout=0.2)(temporal_input_layer)


#硬性条件的全连接层
#####################输入层的属性是5
#####################第一层输出16个属性
########################################dropout同上

demographic_input_layer = Input(shape=(5))
demographic_first_dense=layers.Dense(16)(demographic_input_layer)
demographic_dense = layers.Dense(16)(demographic_first_dense)
demographic_dropout = layers.Dropout(0.2)(demographic_dense)

#这是我们的确诊分支
#连接之前的那一层，有32个单元输出
rnn_c = layers.LSTM(32)(main_rnn_layer)
##################################################并行连接
merge_c = layers.Concatenate(axis=-1)([rnn_c,demographic_dropout])
########################同样是一个全连接层
########################dropot防止过拟合，回忆一下pytorch
dense_c = layers.Dense(128)(merge_c)
dropout_c = layers.Dropout(0.3)(dense_c)
##############################################对执行结果进行激活
##############################################alpha是斜率
#############################################
cases = layers.Dense(1, activation=layers.LeakyReLU(alpha=0.1),name="cases")(dropout_c)

#同理是死亡分支
rnn_f = layers.LSTM(32)(main_rnn_layer)
merge_f = layers.Concatenate(axis=-1)([rnn_f,demographic_dropout])
dense_f = layers.Dense(128)(merge_f)
dropout_f = layers.Dropout(0.3)(dense_f)
fatalities = layers.Dense(1, activation=layers.LeakyReLU(alpha=0.1), name="fatalities")(dropout_f)

########################我们的最终模型
model = Model([temporal_input_layer,demographic_input_layer], [cases,fatalities])

model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=4, verbose=1, factor=0.6),
             EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
model.compile(loss=[tf.keras.losses.MeanSquaredLogarithmicError(),tf.keras.losses.MeanSquaredLogarithmicError()], optimizer="adam")

In [ ]:
history = model.fit([X_temporal_train,X_demographic_train], [Y_cases_train, Y_fatalities_train], 
          epochs = 250, 
          batch_size = 16, 
          validation_data=([X_temporal_test,X_demographic_test],  [Y_cases_test, Y_fatalities_test]), 
          callbacks=callbacks)

# 4. Performance during training

In [ ]:
#################################从图像上看，结果并不好

# 5. 下面这部分最重要！！！！！！！！！！！！！！！！！开始预测！

经历了艰难我们训练出一个模型

In [ ]:
model.load_weights("best_model.h5")

In [ ]:
predictions = model.predict([X_temporal_test,X_demographic_test])

In [ ]:
#########################################没什么好说的。一如既往，我们的输出是一个数组，只有一个元素的数组
#########################################这是我们大概的预测结果


In [ ]:
display_limit = 30
for inputs, pred_cases, exp_cases, pred_fatalities, exp_fatalities in zip(X_temporal_test,predictions[0][:display_limit], Y_cases_test[:display_limit], predictions[1][:display_limit], Y_fatalities_test[:display_limit]):
    print("================================================")
    print(inputs)
    print("Expected cases:", exp_cases, " Prediction:", pred_cases[0], "Expected fatalities:", exp_fatalities, " Prediction:", pred_fatalities[0] )

# 6. 预测模型趋势

我们的预测值可以作为下一次预测的输入变量，不断向下滚动


# 9.已经无法提交了，缩小了训练集，分数太低了

In [ ]:
test_df = gpd.read_file("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
#The country_region got modifying in the enriched dataset by @optimo, 
# so we have to apply the same change to the test Dataframe.
test_df["Country_Region"] = [ row.Country_Region if row.Province_State=="" else row.Country_Region+"_"+row.Province_State for idx,row in test_df.iterrows() ]
test_df.head()

Just need to do this little trick to extract the relevant date and the forecastId and add that to the submission file.